In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
from sklearn import preprocessing
import numpy as np
import time
import os
from PIL import Image
import json
from pathlib import Path
import numpy as np
from sklearn.preprocessing import normalize

In [2]:
# Replace with valid values
ENDPOINT = "https://stylercv.cognitiveservices.azure.com/"
training_key = "fb709acea8b94ff9be5048b8e6c76611"
prediction_key = "b35d976bff3a4d1c9b99c7f493fc7764"
prediction_resource_id = "/subscriptions/6bb19610-9077-414c-971c-e545a2e35961/resourceGroups/styler_0405/providers/Microsoft.CognitiveServices/accounts/stylerCV-Prediction"

In [3]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [4]:
publish_iteration_name = "detectModel-test"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("My Detection Project", domain_id=obj_detection_domain.id)

Creating project...


In [5]:
# Make two tags in the new project

shorts_tag = trainer.create_tag(project.id, "shorts")
shortT_tag = trainer.create_tag(project.id, "shortT")

In [6]:

#建一個w_json的資料夾 裡面放圖片跟json檔
#json_p = 'C:/Users/JIALI/Desktop/styler/Styler_test/w_json/'
json_p = 'C:/Users/JIALI/Desktop/styler/Styler_test/deep_train/'
jList = os.listdir(json_p)
white_image_regions={} #這是一個dic的資料型態
shorts_regions={}
shortT_regions={}

        
for file in jList:
    
    if file.endswith(".jpg"):
        img = Image.open( os.path.join(json_p,file))
        imgSize = img.size  #大小/尺寸
        pw = img.width       #图片的宽
        ph = img.height      #图片的高
        pf = img.format      #图像格式
      #  print(imgSize)
        #print(pw, ph, pf)
    
# Region Left(rl) = X1 / Image Width
# Region Top(rt) = Y1 / Image Height
# Region Width(rw) = (X2 — X1) / image Width
# Region Height(rh)= (Y2 — Y1) / image Height
    
    if file.endswith(".json"):
        with open(os.path.join(json_p,file) , 'r') as reader:
            jf = json.loads(reader.read())
     
           # print(jf['item1']['bounding_box'][0])
            rl=jf['item1']['bounding_box'][0]/pw
            rt = jf['item1']['bounding_box'][1]/ph
            rw=(jf['item1']['bounding_box'][2]-jf['item1']['bounding_box'][0])/pw
            rh=(jf['item1']['bounding_box'][3]-jf['item1']['bounding_box'][1])/ph
            #print(Path(file).stem)
            
            if jf['item1']['category_id']==1:#如果種類是上衣
                print(jf['item1']['bounding_box'])
                shortT_regions[Path(file).stem]=[rl,rt,rw,rh]  #把東西丟到這個dic裡    
            if jf['item1']['category_id']==7: #如果種類是短褲
                shorts_regions[Path(file).stem]=[rl,rt,rw,rh]  
          #  print (white_image_regions)
  
    

[2, 199, 466, 788]
[93, 177, 407, 592]
[87, 0, 466, 623]
[0, 62, 466, 623]
[2, 78, 465, 618]
[118, 165, 368, 504]
[31, 145, 425, 717]
[14, 0, 466, 537]
[1, 35, 451, 435]
[144, 142, 426, 429]
[66, 41, 470, 388]
[178, 168, 470, 462]
[0, 0, 383, 511]
[110, 0, 443, 322]
[134, 69, 369, 311]
[60, 149, 364, 503]
[0, 0, 467, 400]
[2, 86, 467, 503]
[75, 184, 410, 476]
[0, 0, 434, 453]
[347, 190, 704, 554]
[127, 262, 656, 741]
[326, 199, 691, 584]
[200, 275, 614, 639]
[101, 170, 338, 397]
[165, 284, 592, 679]
[75, 172, 306, 392]
[168, 275, 548, 651]
[108, 279, 643, 758]
[163, 279, 570, 682]
[140, 356, 463, 824]
[166, 8, 347, 203]
[239, 240, 466, 623]
[197, 0, 467, 350]
[128, 186, 441, 570]
[70, 171, 533, 585]
[39, 230, 438, 610]
[90, 224, 417, 657]
[119, 195, 470, 546]
[145, 217, 521, 705]
[64, 173, 369, 542]


In [7]:
# Update this with the path to where you downloaded the images.
base_image_location = "C:/Users/JIALI/Desktop/styler/Styler_test/deep_train/"

# Go through the data table above and create the images
print ("Adding images...")
tagged_images_with_regions = []

for file_name in shortT_regions.keys():
    x,y,w,h =shortT_regions[file_name]
    regions = [ Region(tag_id=shortT_tag.id, left=x,top=y,width=w,height=h) ]

    with open(base_image_location  + file_name + ".jpg", mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

for file_name in shorts_regions.keys():
    x,y,w,h =shorts_regions[file_name]
    regions = [ Region(tag_id=shorts_tag.id, left=x,top=y,width=w,height=h) ]

    with open(base_image_location + file_name + ".jpg", mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


In [8]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training sta

In [9]:
# The iteration is now trained. Publish it to the project endpoint
#trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
trainer.publish_iteration(project.id, iteration.id,publish_iteration_name ,  prediction_resource_id)

print ("Done!")

Done!


In [10]:
# Now there is a trained endpoint that can be used to make a prediction

# Open the sample image and get back the prediction results.
with open("C:/Users/JIALI/Desktop/styler/Styler_test/w_json/000235.jpg", mode="rb") as test_data:
    results = predictor.detect_image(project.id, publish_iteration_name, test_data)

# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))

	shortT: 19.07% bbox.left = 0.00, bbox.top = 0.36, bbox.width = 0.98, bbox.height = 0.60
	shortT: 2.69% bbox.left = 0.58, bbox.top = 0.03, bbox.width = 0.42, bbox.height = 0.54
	shortT: 2.58% bbox.left = 0.06, bbox.top = 0.00, bbox.width = 0.54, bbox.height = 0.51
	shortT: 1.24% bbox.left = 0.00, bbox.top = 0.06, bbox.width = 1.00, bbox.height = 0.57
	shortT: 0.96% bbox.left = 0.15, bbox.top = 0.31, bbox.width = 0.07, bbox.height = 0.07
	shortT: 0.81% bbox.left = 0.06, bbox.top = 0.31, bbox.width = 0.08, bbox.height = 0.07
	shortT: 0.72% bbox.left = 0.70, bbox.top = 0.36, bbox.width = 0.30, bbox.height = 0.61
	shortT: 0.69% bbox.left = 0.07, bbox.top = 0.36, bbox.width = 0.08, bbox.height = 0.08
	shortT: 0.65% bbox.left = 0.92, bbox.top = 0.31, bbox.width = 0.08, bbox.height = 0.07
	shortT: 0.62% bbox.left = 0.92, bbox.top = 0.20, bbox.width = 0.08, bbox.height = 0.07
	shortT: 0.61% bbox.left = 0.77, bbox.top = 0.53, bbox.width = 0.23, bbox.height = 0.25
	shortT: 0.60% bbox.left = 0.14